# Quickstart

This notebook contains the code examples from the [readme](../README.md).

In [ ]:
import enact
import dataclasses

# Allow re-registration of resources to avoid errors when re-running cells.
enact.Registry.get().allow_reregistration = True

## Defining and storing custom resources

In [ ]:
@enact.register
@dataclasses.dataclass
class MyResource(enact.Resource):
  x: int
  y: float

with enact.Store() as store:
  ref = enact.commit(MyResource(42, 69.0))
  print(ref.id)
  print(ref.get())

## Invoking resources

In [ ]:
@enact.typed_invokable(input_type=enact.NoneResource, output_type=MyResource)
class MyInvokable(enact.Invokable):

  def call(self):
    return MyResource(42, 69.0)

with store:
  my_invokable = MyInvokable()
  # Simple execution:
  print(my_invokable())  # Prints "MyResource(x=42, y=69.0)".
  # Tracked execution:
  invocation = my_invokable.invoke()
  enact.pprint(invocation)

## Creating UIs

In [ ]:
with store:
  ref = enact.commit(my_invokable)
  enact.GUI(ref).launch(share=True)

### Requesting inputs and replaying invocations

In [ ]:
@enact.typed_invokable(input_type=enact.NoneResource, output_type=MyResource)
class SampleFromHuman(enact.Invokable):

  def call(self):
    request_int = enact.RequestInput(enact.Int)
    request_float = enact.RequestInput(enact.Float)
    return MyResource(
      x=request_int(enact.Str('Please provide an x-value for MyResource.')),
      y=request_float(enact.Str('Please provide a y-value for MyResource.')))

with store:
  h = SampleFromHuman()
  # Run until first input request.
  invocation = h.invoke()
  # Access InputRequest exception.
  input_request = invocation.response().raised()
  print(input_request.input())  # Prints 'Please provide an x-value ...'.
  # Run until second input request.
  invocation = input_request.continue_invocation(invocation, enact.Int(42))
  # Access InputRequest exception.
  input_request = invocation.response().raised()
  print(input_request.input())  # Prints 'Please provide a y-value ...'.
  # Run until completion.
  invocation = input_request.continue_invocation(invocation, enact.Float(69.0))
  print(invocation.response().output())  # Prints 'MyResource(x=42, y=69.0)'.

The above is roughly equivalent to:

In [ ]:
with store:
  # Run until first exception.
  invocation = h.invoke()
  def override_exception(exc_ref):
    if exc_ref().requested_type == enact.Int:
      return enact.Int(42)
    if exc_ref().requested_type == enact.Float:
      return enact.Float(69.0)
  # Inject first value and run until second exception.
  invocation = invocation.replay(override_exception)
  # Inject second value and run until completion.
  invocation = invocation.replay(override_exception)
  print(invocation.response().output())  # Prints 'MyResource(x=42, y=69.0)'.

UIs can handle input requests automatically. The required types needs to be
specified on launch.

In [ ]:
with store:
  ref = enact.commit(h)
  enact.GUI(ref, input_required_inputs=[enact.Int, enact.Float]).launch(
    share=True)